In [ ]:
from utils.animate_ball3 import BreakoutGame
%matplotlib tk
game = BreakoutGame(framesize=(64,64), xy=[[2,2],[5,2],[5,2],[5,2]], color=255, direction=[[1,1],[1,1],[1,2],[1,3]],nr_sim_steps=10,obstacle_color=200,destructible_obstacles=True,boundary_color=50,player_color=255,nr_balls=4)

for i in range(20):
    game.gs.add_obstacles([[i+10,29]])
game.main_game_loop()

In [ ]:
import gymnasium as gym
import numpy as np
from utils.dqn_agent import DQN as DQNAgent
from utils.animate_ball3 import BreakoutGame
import random
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import torch
def save_model(agent):
    torch.save(agent.model.state_dict(), "model.pth")
def load_model(path="model.pth",action_size=3,state_size=32*32*3):
    agent = DQNAgent(state_size, action_size)
    agent.model.load_state_dict(torch.load(path))
    return agent


writer = SummaryWriter("runs/"+str(datetime.now().strftime("%Y%m%d-%H%M%S")))
# We create our gym environment 
#env = BreakoutGame(framesize=(64,64), xy=[[5,2]], color=255, direction=[[1,1]],nr_sim_steps=10,obstacle_color=200,destructible_obstacles=False,boundary_color=50,player_color=255,nr_balls=1)
# We get the shape of a state and the actions space size
state_size = 32*32*3#doesnt matter
action_size = 3#left,right,none
# Number of episodes to run
n_episodes = 15000

# Max iterations per epiode
max_iteration_ep = 500
# We define our agent
agent = DQNAgent(state_size, action_size)
agent.lr=1e-7
agent.gamma=0.99
agent.exploration_proba=1
agent.exploration_proba_decay=0.001
agent.batch_size=16
agent.verbose=True
batch_size=agent.batch_size
total_steps = 0
min_loss=1e10

# We iterate over episodes
for e in tqdm(range(n_episodes)):
    # We initialize the first state and reshape it to fit 
    #  with the input layer of the DNN
    dirx = random.random()*8-4
    diry = random.random()*8-4#
    dirx=random.choice([-2,-1,0,1,2])
    diry=random.choice([-2,-1,1,2])
    xy=[random.randint(2,28),random.randint(10,28)]

    env = BreakoutGame(framesize=(32,32), xy=[xy], color=255, direction=[[dirx,diry]],nr_sim_steps=10,obstacle_color=200,destructible_obstacles=False,boundary_color=50,player_color=255,nr_balls=1)

    current_state = env.get_initial_state(nr_frames=3)#runs the game for 3 frames
    current_state = np.array(current_state)


    for step in range(max_iteration_ep):
        
        total_steps = total_steps + 1
        # the agent computes the action to perform
        action = agent.compute_action(current_state)
        if action==0:
            action=[0,0]
        if action==1:
            action=[2,0]
        if action==2:
            action=[-2,0]
        # the envrionment runs the action and returns
        # the next state, a reward and whether the agent is done

        ##next_state, reward, done, _,_ = env.step(action)
        next_state, reward, done = env.rl_refresh_frame(action,current_state)
        
        #print(current_state,step,reward)
        next_state = np.array(next_state)
        #print(next_state)
        
        # We sotre each experience in the memory buffer
        #print("current_state",current_state,"action",action,"reward",reward,"next_state",next_state,"done",done)
        agent.store_episode(current_state, action, reward, next_state, done)
        
        # if the episode is ended, we leave the loop after
        # updating the exploration probability
        if done:
            agent.update_exploration_probability()
            break
        current_state = next_state
    # if the have at least batch_size experiences in the memory buffer
    # than we tain our model
    if total_steps >= batch_size:
        loss=agent.train()
        try:
            writer.add_scalar("Loss/train", loss, e)
        except:
            pass
        if loss<min_loss:
            min_loss=loss
            save_model(agent)
            print("saved model at episode",e)


In [1]:
import torch
from utils.dqn_agent import DQN as DQNAgent
from utils.animate_ball3 import BreakoutGame
import numpy as np

#save_model(agent)
#agent=load_model()


/var/folders/gj/b_6qbpf106vgb2f614dhqxt40000gn/T/ipykernel_5327/3096028262.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  agent.model.load_state_dict(torch.load(path))


In [5]:
import matplotlib.pyplot as plt
%matplotlib tk
plt.ion()
#fig = plt.figure(1)
fig, ax = plt.subplots(1,2)
env = BreakoutGame(framesize=(32,32), xy=[[20,20]], color=255, direction=[[3,1]],nr_sim_steps=10,obstacle_color=200,destructible_obstacles=False,boundary_color=50,player_color=255,nr_balls=1)
current_state = env.get_initial_state(nr_frames=3)#runs the game for 3 frames
current_state = np.array(current_state)
action=[0,0]
frame_nr=0
while env.has_ended==False:
    next_state, reward, done = env.rl_refresh_frame(action,current_state)
    action = agent.compute_action(next_state,explore=False)
    if action==0:
        action=[0,0]
    if action==1:
        action=[2,0]
    if action==2:
        action=[-2,0]
    env.gs.player_direction=action
    frame=env.refresh_frame()

    
    ax[0].imshow(np.flip(frame.T,0))
    ax[0].set_title("RL Agent")
    if frame_nr>0:
        pred_frame=agent.pred_frame
        pred_frame=pred_frame.squeeze(0)
        pred_frame=pred_frame.cpu().detach().numpy()
        print(pred_frame)
        ax[1].imshow(np.flip(pred_frame.T,0))
        ax[1].set_title("Predicted Frame")
#
    plt.pause(0.03)
    for a in ax:
        a.cla()
    #plt.clf()
    frame_nr+=1

tensor([[[[0.4547, 0.3922, 0.2518,  ..., 0.3877, 0.4548, 0.4558],
          [0.3888, 0.4192, 0.3985,  ..., 0.3978, 0.4217, 0.4979],
          [0.4786, 0.3291, 0.3775,  ..., 0.4081, 0.4521, 0.4379],
          ...,
          [0.4541, 0.4763, 0.4717,  ..., 0.3590, 0.4404, 0.5535],
          [0.5235, 0.5291, 0.4977,  ..., 0.2998, 0.2711, 0.3895],
          [0.4768, 0.4201, 0.4588,  ..., 0.3156, 0.4268, 0.4427]]]],
       grad_fn=<SigmoidBackward0>)
tensor([[[[0.4581, 0.3819, 0.2601,  ..., 0.3772, 0.4529, 0.4615],
          [0.3783, 0.4337, 0.4248,  ..., 0.4111, 0.4375, 0.4937],
          [0.4769, 0.3395, 0.3646,  ..., 0.4081, 0.4608, 0.4483],
          ...,
          [0.4821, 0.4690, 0.4493,  ..., 0.3866, 0.4606, 0.5447],
          [0.5353, 0.5510, 0.4895,  ..., 0.3062, 0.2967, 0.4228],
          [0.4762, 0.4354, 0.4755,  ..., 0.3308, 0.4480, 0.4590]]]],
       grad_fn=<SigmoidBackward0>)
[[[0.4581057  0.38190946 0.26005337 ... 0.3772411  0.45289618 0.4614986 ]
  [0.37828225 0.43373883 0.4

In [ ]:
agent.pred_frame